# 1. The Dataset

- [GTZAN](https://huggingface.co/datasets/marsyas/gtzan) dataset: 1,000 songs for music genre classification

- Each song is a 30-second clip from one of 10 genres of music, spanning disco to metal.

In [1]:
%pip install accelerate datasets gradio evaluate soundfile librosa numpy transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Get the audio files and their corresponding labels from the Hugging Face Hub
# One of the recordings in GTZAN is corrupted, so it’s been removed from the dataset. That’s why we have 999 examples instead of 1,000.
from datasets import load_dataset
gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [3]:
# Create validation set
gtzan = gtzan['train'].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [4]:
# Look at one of the audio file
gtzan["train"][0]

{'file': '/mnt/data1/kimjaewon/pkgs/.cache/huggingface/datasets/downloads/extracted/abcff08a49abd5659eddadaa09daa2404216ec07f98224f10a63db75996935d1/genres/pop/pop.00098.wav',
 'audio': {'path': '/mnt/data1/kimjaewon/pkgs/.cache/huggingface/datasets/downloads/extracted/abcff08a49abd5659eddadaa09daa2404216ec07f98224f10a63db75996935d1/genres/pop/pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229]),
  'sampling_rate': 22050},
 'genre': 7}

In [5]:
# Use the int2str() method of the genre feature to map integers to human-readable names
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'pop'

# 2. Picking a pretrained model for audio classification
- Although models like Wav2Vec2 and HuBERT are very popular, we'll use a model called DistilHuBERT
- I've picked ntu-spml/distilhubert

# 3. Preprocessing the data
- Speech models require the input to be encoded in a format that the model can process
- In Transformers, the conversion from audio to the input format is handled by the feature extractor of the model
- Transformers provides a convenient AutoFeatureExtractor class that can automatically select the correct feature extractor for a given model.

In [6]:
# Instantiating the feature extractor for DistilHuBERT from the pre-trained checkpoint
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [7]:
# Resample the audio fil to 16,000Hz
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [8]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [9]:
gtzan["train"][0]

{'file': '/mnt/data1/kimjaewon/pkgs/.cache/huggingface/datasets/downloads/extracted/abcff08a49abd5659eddadaa09daa2404216ec07f98224f10a63db75996935d1/genres/pop/pop.00098.wav',
 'audio': {'path': '/mnt/data1/kimjaewon/pkgs/.cache/huggingface/datasets/downloads/extracted/abcff08a49abd5659eddadaa09daa2404216ec07f98224f10a63db75996935d1/genres/pop/pop.00098.wav',
  'array': array([ 0.08735093,  0.20183384,  0.47908676, ..., -0.1874318 ,
         -0.23294398, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

In [10]:
# Normalize the audio data, by rescaling each sample to zero mean and unit variance,
# a process called feature scaling

import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


In [11]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -6.72e-09, Variance: 1.0


In [12]:
# Define the max duration of the input audio
max_duration = 30.0

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [13]:
# Use map() method to apply the whole preprocess into dataset
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [14]:
# Rename the genre column to label: enable the Trainer to process the class labels
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [15]:
# Obtain label mappings from the dataset
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

# 4. Fine-tuning the model
- Use Trainer class from transformers

In [16]:
# Load a model
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.weight', 'classifier.bias', 'projector.weight', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Log in Hugging Face Hub
from huggingface_hub import notebook_login

notebook_login()

In [33]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import torch
torch.cuda.set_device(2)
print(torch.cuda.current_device())

2


In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
device
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [44]:
from transformers import TrainingArguments, get_linear_schedule_with_warmup, Trainer
import torch.optim as optim
import numpy as np
import evaluate
import torch

# Define the training arguments
model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
    seed=42,
    lr_scheduler_type="linear",
)

# Define custom Adam optimizer with specific betas and epsilon values
class CustomTrainer(Trainer):
    def create_optimizer_and_scheduler(self, num_training_steps: int):
        self.optimizer = optim.Adam(
            self.model.parameters(),
            lr=self.args.learning_rate,
            betas=(0.9, 0.999),
            eps=1e-08
        )

        self.lr_scheduler = get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=int(num_training_steps * 0.1),
            num_training_steps=num_training_steps
        )

# Define the learning rate scheduler
total_steps = len(gtzan_encoded["train"]) * num_train_epochs // batch_size

# Define the metrics
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

# Instantiate the Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,  # Assuming you've already defined this
    compute_metrics=compute_metrics
)

# Empty the cache
torch.cuda.empty_cache()

# Train the model
trainer.train()


AttributeError: can't set attribute 'device'

In [28]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

In [ ]:
trainer.push_to_hub(**kwargs)